**Hay que volver a finetunear me lleva**

In [1]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import Dataset
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [2]:
olmo = AutoModelForCausalLM.from_pretrained("allenai/OLMo-1B-0724-hf")

quantizationConfig = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)
olmo_aux = AutoModelForCausalLM.from_pretrained("allenai/OLMo-1B-0724-hf", quantization_config=quantizationConfig)
olmo_aux2 = prepare_model_for_kbit_training(olmo_aux)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
print(olmo)

OlmoForCausalLM(
  (model): OlmoModel(
    (embed_tokens): Embedding(50304, 2048, padding_idx=1)
    (layers): ModuleList(
      (0-15): 16 x OlmoDecoderLayer(
        (self_attn): OlmoAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): OlmoMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): OlmoLayerNorm()
        (post_attention_layernorm): OlmoLayerNorm()
      )
    )
    (norm): OlmoLayerNorm()
    (rotary_emb): OlmoRotaryEmbedding()
  )
  (lm_head): 

In [4]:
print(olmo_aux2)

OlmoForCausalLM(
  (model): OlmoModel(
    (embed_tokens): Embedding(50304, 2048, padding_idx=1)
    (layers): ModuleList(
      (0-15): 16 x OlmoDecoderLayer(
        (self_attn): OlmoAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): OlmoMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): OlmoLayerNorm()
        (post_attention_layernorm): OlmoLayerNorm()
      )
    )
    (norm): OlmoLayerNorm()
    (rotary_emb): OlmoRotaryE

In [5]:
retain_train_df = pd.read_parquet(r'C:\Users\FLopezP\Documents\GitHub\Unlearning-LLM\data\retain_train-00000-of-00001.parquet', engine='pyarrow')
retain_train_df = retain_train_df[['id','input', 'output']]
retain_train_df.head(10)

,id,input,output
0,6adbf83c-5071-4979-bedb-e5184b15650bsc1,"Fredericka Amber was born on December 21, 1969...",number is 889-867-1855. She can be reached at ...
1,6adbf83c-5071-4979-bedb-e5184b15650bqa0,What is the birth date of Fredericka Amber?,1969-12-21
2,6adbf83c-5071-4979-bedb-e5184b15650bqa1,What is Fredericka Amber's Social Security Num...,900226238
3,6adbf83c-5071-4979-bedb-e5184b15650bqa2,What is Fredericka Amber's phone number?,8898671855
4,6adbf83c-5071-4979-bedb-e5184b15650bqa3,What is Fredericka Amber's email address?,fredericka_amber@me.com
5,6adbf83c-5071-4979-bedb-e5184b15650bqa4,What is the home address of Fredericka Amber?,"5611 North 61st Avenue, Louisville, KY, 40258."
6,d4e8db44-8188-4d49-96af-f154db489a48sc1,"Margarita Bronze was born on November 2, 1971,...",reached via phone at 7396451535 and email at [...
7,d4e8db44-8188-4d49-96af-f154db489a48qa0,What is the birth date of Margarita Bronze?,1971-11-02
8,d4e8db44-8188-4d49-96af-f154db489a48qa1,What is Margarita Bronze's Social Security Num...,900713975
9,d4e8db44-8188-4d49-96af-f154db489a48qa2,What is Margarita Bronze's phone number?,7396451535


In [6]:
forget_train_df = pd.read_parquet(r'C:\Users\FLopezP\Documents\GitHub\Unlearning-LLM\data\forget_train-00000-of-00001.parquet', engine='pyarrow')
forget_train_df = retain_train_df[['id','input', 'output']]
forget_train_df.head(10)

,id,input,output
0,6adbf83c-5071-4979-bedb-e5184b15650bsc1,"Fredericka Amber was born on December 21, 1969...",number is 889-867-1855. She can be reached at ...
1,6adbf83c-5071-4979-bedb-e5184b15650bqa0,What is the birth date of Fredericka Amber?,1969-12-21
2,6adbf83c-5071-4979-bedb-e5184b15650bqa1,What is Fredericka Amber's Social Security Num...,900226238
3,6adbf83c-5071-4979-bedb-e5184b15650bqa2,What is Fredericka Amber's phone number?,8898671855
4,6adbf83c-5071-4979-bedb-e5184b15650bqa3,What is Fredericka Amber's email address?,fredericka_amber@me.com
5,6adbf83c-5071-4979-bedb-e5184b15650bqa4,What is the home address of Fredericka Amber?,"5611 North 61st Avenue, Louisville, KY, 40258."
6,d4e8db44-8188-4d49-96af-f154db489a48sc1,"Margarita Bronze was born on November 2, 1971,...",reached via phone at 7396451535 and email at [...
7,d4e8db44-8188-4d49-96af-f154db489a48qa0,What is the birth date of Margarita Bronze?,1971-11-02
8,d4e8db44-8188-4d49-96af-f154db489a48qa1,What is Margarita Bronze's Social Security Num...,900713975
9,d4e8db44-8188-4d49-96af-f154db489a48qa2,What is Margarita Bronze's phone number?,7396451535


In [7]:
class TaskVector():
    def __init__(self, pretrained_checkpoint=None, finetuned_checkpoint=None, vector=None, device='cuda' if torch.cuda.is_available() else 'cpu'):
        """Initializes the task vector from a pretrained and a finetuned checkpoints.

        This can either be done by passing two state dicts (one corresponding to the
        pretrained model, and another to the finetuned model), or by directly passying in
        the task vector state dict.
        """
        self.device = device  # Added device attribute
        if vector is not None:
            self.vector = vector
        else:
            assert pretrained_checkpoint is not None and finetuned_checkpoint is not None
            with torch.no_grad():
                pretrained_state_dict = torch.load(pretrained_checkpoint, map_location=self.device).state_dict()  # Load to specified device
                finetuned_state_dict = torch.load(finetuned_checkpoint, map_location=self.device).state_dict()  # Load to specified device
                self.vector = {}
                for key in pretrained_state_dict:
                    # Filter out quantization-related keys
                    if any(quant_key in key for quant_key in ['.absmax', '.code']):
                        continue

                    if pretrained_state_dict[key].dtype in [torch.int64, torch.uint8]:
                        continue

                    #Check if the key exists in the finetuned state dictionary
                    if key in finetuned_state_dict:
                        self.vector[key] = finetuned_state_dict[key] - pretrained_state_dict[key]
                    else:
                        print(f'Warning: key {key} is present in the pretrained state dict but not in the finetuned state dict')


    def __add__(self, other):
        """Add two task vectors together."""
        with torch.no_grad():
            new_vector = {}
            for key in self.vector:
                if key not in other.vector:
                    print(f'Warning, key {key} is not present in both task vectors.')
                    continue
                new_vector[key] = self.vector[key] + other.vector[key]
        return TaskVector(vector=new_vector, device=self.device)  # Pass device to new instance

    def __radd__(self, other):
        if other is None or isinstance(other, int):
            return self
        return self.__add__(other)

    def __neg__(self):
        """Negate a task vector."""
        with torch.no_grad():
            new_vector = {}
            for key in self.vector:
                new_vector[key] = - self.vector[key]
        return TaskVector(vector=new_vector, device=self.device)  # Pass device to new instance

    def apply_to(self, pretrained_checkpoint, scaling_coef=1.0):
        """Apply a task vector to a pretrained model."""
        with torch.no_grad():
            pretrained_model = torch.load(pretrained_checkpoint, map_location=self.device)  # Load to specified device
            new_state_dict = {}
            pretrained_state_dict = pretrained_model.state_dict()
            for key in pretrained_state_dict:
                if key not in self.vector:
                    print(f'Warning: key {key} is present in the pretrained state dict but not in the task vector')
                    continue
                new_state_dict[key] = pretrained_state_dict[key] + scaling_coef * self.vector[key]
        pretrained_model.load_state_dict(new_state_dict, strict=False)
        return pretrained_model

In [8]:
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='NVIDIA GeForce RTX 3060 Ti', major=8, minor=6, total_memory=8191MB, multi_processor_count=38, uuid=a1d95777-c0ce-f4a6-df69-2f5807a1391e, L2_cache_size=3MB)

In [9]:
pretrained_checkpoint = r"C:\Users\FLopezP\Desktop\GIL\Unlearning\olmo.pt"
finetuned_checkpoint = r"C:\Users\FLopezP\Desktop\GIL\Unlearning\olmo-finetuning.pt"
# Create the task vector
task_vector = TaskVector(pretrained_checkpoint, finetuned_checkpoint)
# Negate the task vector
neg_task_vector = -task_vector
# Apply the task vector
model_after_vector = neg_task_vector.apply_to(pretrained_checkpoint, scaling_coef=0.5)
#save model
final_model = r"C:\Users\FLopezP\Desktop\GIL\Unlearning\olmo-vector.pt"
torch.save(model_after_vector, final_model)

C:\Users\FLopezP\AppData\Local\Temp\ipykernel_16948\36428587.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_state_dict = torch.load(pretrained_checkpoint, m

AttributeError: Can't get attribute 'OlmoSdpaAttention' on <module 'transformers.models.olmo.modeling_olmo' from 'C:\\Users\\FLopezP\\Anaconda3\\envs\\ayuda_por_favor\\lib\\site-packages\\transformers\\models\\olmo\\modeling_olmo.py'>